In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw
from sklearn.utils import class_weight
from datetime import datetime

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
df_train = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/df_train_raMSIn4nonInDI.csv').set_index('pixel_id')
df_train.shape
df_ext = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/df_ext_raMSIn4nonInDI.csv').set_index('pixel_id')
df_ext.shape
df_FNA = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/df_FNA_raMSIn4nonInDI.csv').set_index('pixel_id')
df_FNA.shape

(88701, 18)

# 1. Data Pre-Processing

In [3]:
df_train = df_train.reset_index()
df_ext = df_ext.reset_index()
df_FNA = df_FNA.reset_index()

In [4]:
df_train

,pixel_id,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,280.2361,241.2173,311.2228,339.1996,353.2005,325.1842,250.1449,514.2846,265.1478,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,69090.671875,651.139648,1765.094360,0.0,2001.672241,23738.072266,2756.949280,1833.859131,6426.550781,1359.884644,0.0,22193.466797,5444.159668,52111.000000,10417.782227,0.000000,38821.574219,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,115699.093750,1621.182007,2892.591553,0.0,2424.003906,35038.671875,1765.605469,1439.742432,5823.882324,1761.496948,0.0,33849.113281,7857.419922,83056.984375,19751.246094,0.000000,54593.625000,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,62308.257812,700.977356,2225.103027,0.0,2032.922119,17089.125000,1900.651733,948.719055,6255.112793,1269.849487,0.0,22055.828125,5718.301758,45621.710938,14473.411133,0.000000,37252.769531,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,55687.476562,547.317627,2827.721436,0.0,2229.916992,15492.205078,2011.564331,1006.617981,5697.002930,1637.160278,0.0,20968.199219,4118.433105,40787.113281,13347.621094,0.000000,34972.964844,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,29014.601562,813.711731,1611.982788,0.0,2252.401855,9364.838867,1967.201538,1989.947754,7003.378418,1039.213745,0.0,12019.461914,3703.121338,21758.828125,9052.196289,0.000000,19464.642578,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,6974.704590,0.000000,3595.426758,0.0,1217.228271,1812.482544,1256.135132,2806.170410,3905.608887,0.000000,0.0,4688.658203,970.321777,6651.193359,6830.675293,1873.401733,2161.815186,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,5604.379395,0.000000,2122.539551,0.0,2125.813721,2162.781006,1358.445190,3326.609375,3403.044434,970.017761,0.0,5139.420410,1027.451416,6818.199219,14570.761719,0.000000,1393.699585,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,6498.166504,1333.430664,2093.726318,0.0,1770.959473,1309.870850,2459.356689,2415.567139,4329.395020,0.000000,0.0,4925.088867,1573.964844,6854.788086,11204.238281,0.000000,0.000000,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,8075.770508,0.000000,1721.714355,0.0,1023.813782,1725.560425,2080.640381,2060.364746,4167.682129,0.000000,0.0,4179.726562,0.000000,6759.228027,13443.030273,0.000000,2786.455078,1


In [5]:
df_train["Sample"] = 0
df_train["Row"] = 0
df_train["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(df_train["pixel_id"])):
    Sample.append("_".join(str(df_train["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(df_train["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(df_train["pixel_id"][i]).split("_")[-2]))
df_train["Sample"] = Sample
df_train["Row"] = Row
df_train["Scan"] = Scan

In [6]:
df_ext["Sample"] = 0
df_ext["Row"] = 0
df_ext["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(df_ext["pixel_id"])):
    Sample.append("_".join(str(df_ext["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(df_ext["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(df_ext["pixel_id"][i]).split("_")[-2]))
df_ext["Sample"] = Sample
df_ext["Row"] = Row
df_ext["Scan"] = Scan

In [7]:
df_FNA["Sample"] = 0
df_FNA["Row"] = 0
df_FNA["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(df_FNA["pixel_id"])):
    Sample.append("_".join(str(df_FNA["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(df_FNA["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(df_FNA["pixel_id"][i]).split("_")[-2]))
df_FNA["Sample"] = Sample
df_FNA["Row"] = Row
df_FNA["Scan"] = Scan

In [8]:
pop_column = df_train.pop('Scan')
df_train.insert(1, 'Scan', pop_column)
pop_column = df_train.pop('Row')
df_train.insert(1, 'Row', pop_column)
pop_column = df_train.pop('Sample')
df_train.insert(1, 'Sample', pop_column)

pop_column = df_ext.pop('Scan')
df_ext.insert(1, 'Scan', pop_column)
pop_column = df_ext.pop('Row')
df_ext.insert(1, 'Row', pop_column)
pop_column = df_ext.pop('Sample')
df_ext.insert(1, 'Sample', pop_column)

pop_column = df_FNA.pop('Scan')
df_FNA.insert(1, 'Scan', pop_column)
pop_column = df_FNA.pop('Row')
df_FNA.insert(1, 'Row', pop_column)
pop_column = df_FNA.pop('Sample')
df_FNA.insert(1, 'Sample', pop_column)

In [9]:
df_train = df_train.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ext = df_ext.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_FNA = df_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [10]:
raMSIn = pd.concat([df_train, df_ext, df_FNA])

In [11]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [12]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,...,280.2361,241.2173,311.2228,339.1996,353.2005,325.1842,250.1449,514.2846,265.1478,type
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,470.536774,0.000000,3152.019287,872.196777,1796.736816,1794.692261,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,5024.136719,2248.198242,1412.507568,0
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,5,0.000000,677.591492,3041.586182,1622.129883,1665.447632,1596.109985,...,0.000000,0.000000,742.270203,0.0,0.000000,519.868652,7385.251953,1403.369507,1458.948853,0
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,6,0.000000,880.095154,3117.169434,1102.376709,1464.021362,1421.844604,...,0.000000,1117.611572,0.000000,0.0,0.000000,844.534180,8289.688477,2273.504150,1149.217773,0
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,194,0.000000,8586.939453,6283.457520,14824.247070,1961.100708,13860.533203,...,891.801941,7787.978027,8383.257812,0.0,0.000000,0.000000,27578.626953,11022.319336,2026.847168,0
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,195,0.000000,7384.484375,7803.814941,17598.421875,1127.184814,14293.389648,...,1293.782104,7533.042480,8323.607422,0.0,739.255371,0.000000,29367.962891,17072.466797,2953.884521,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,144,0.000000,966.017944,1684.391724,1427.662720,2199.262695,450.825867,...,5105.963867,0.000000,0.000000,0.0,0.000000,0.000000,2754.986572,519.116028,0.000000,0
14821,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,145,0.000000,943.075989,1974.694214,2367.618164,1466.694336,0.000000,...,5540.858887,693.795471,502.354218,0.0,0.000000,0.000000,3987.395264,1199.406250,0.000000,0
14822,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,146,0.000000,854.914978,2245.246094,2563.291504,885.667908,0.000000,...,3985.450684,867.564880,561.708557,0.0,0.000000,0.000000,3384.096436,553.125732,0.000000,0
14823,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,118,177,0.000000,5242.995117,0.000000,5321.240234,687.377136,2062.500000,...,564.445435,4923.465332,1969.879028,0.0,0.000000,0.000000,15790.973633,0.000000,0.000000,0


In [13]:
HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_

,pixel_id,Sample,Row,Scan,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,...,280.2361,241.2173,311.2228,339.1996,353.2005,325.1842,250.1449,514.2846,265.1478,type
0,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,2,68,5669.102051,0.000000,8025.071289,0.000000,3403.159180,0.000000,...,4007.489258,1743.979492,0.0,3396.341797,0.000000,4282.301270,13494.002930,2880.299805,0.000000,1
1,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,2,69,6002.591797,0.000000,8084.173340,0.000000,2187.784912,0.000000,...,4147.172852,944.711487,0.0,4740.611328,0.000000,5116.940918,14259.791992,2648.131348,1155.229858,1
2,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,2,70,4276.868164,0.000000,7036.358887,0.000000,2885.473145,0.000000,...,4715.352051,917.910583,0.0,4074.914062,0.000000,5073.931152,13459.212891,3284.619385,2230.146240,1
3,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,2,71,4584.000977,0.000000,7888.710938,0.000000,2363.930664,0.000000,...,4351.367188,0.000000,0.0,5088.734863,0.000000,5204.424316,11998.317383,1783.281250,0.000000,1
4,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,2,110,5826.635254,1136.738892,5014.947754,0.000000,2347.218994,0.000000,...,3871.852783,1890.988892,0.0,4793.948730,0.000000,8093.779297,13069.095703,2033.081543,2397.725342,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8831,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,94,159,7927.900879,0.000000,5764.550293,0.000000,2365.132080,1158.364868,...,6652.764648,0.000000,0.0,3319.478760,0.000000,7015.848633,9128.623047,1215.611084,1709.268188,1
8832,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,94,160,8617.798828,1027.628174,5076.562500,0.000000,3366.364746,1954.081909,...,8128.616699,0.000000,0.0,4610.856934,0.000000,7748.165527,9759.905273,0.000000,2443.068115,1
8833,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,94,161,8790.328125,0.000000,6157.842773,1013.109314,3151.643311,0.000000,...,8781.387695,1162.241333,0.0,4079.873779,1369.290771,7935.246094,9359.820312,1864.319946,3263.484375,1
8834,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,94,162,12574.975586,0.000000,5609.024414,0.000000,3630.000244,1812.485840,...,9877.017578,0.000000,0.0,5693.137207,1218.620728,11472.702148,14537.357422,1475.821777,3146.960205,1


In [14]:
sample_list_ = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_]
name_list = ["HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"]

# 2. Visualization

In [15]:
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/"

if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)


In [16]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm
import numpy as np
from matplotlib.colors import ListedColormap
cmap = cm.binary

def rgb_white2alpha(rgb, ensure_increasing=False):

    # The most transparent alpha we can use is given by the min of RGB
    # Convert it from saturation to opacity
    alpha = 1. - np.min(rgb, axis=1)
    if ensure_increasing:
        # Let's also ensure the alpha value is monotonically increasing
        a_max = alpha[0]
        for i, a in enumerate(alpha):
            alpha[i] = a_max = np.maximum(a, a_max)
    alpha = np.expand_dims(alpha, -1)
    # Rescale colors to discount the white that will show through from transparency
    rgb = (rgb + alpha - 1) / alpha
    # Concatenate our alpha channel
    return np.concatenate((rgb, alpha), axis=1)
    

def cmap_white2alpha(name, ensure_increasing=False, register=True):

    # Fetch the cmap callable
    cmap = plt.get_cmap(name)
    # Get the colors out from the colormap LUT
    rgb = cmap(np.arange(cmap.N))[:, :3]  # N-by-3
    # Convert white to alpha
    rgba = rgb_white2alpha(rgb, ensure_increasing=ensure_increasing)
    # Create a new Colormap object
    cmap_alpha = matplotlib.colors.ListedColormap(rgba, name=name + "_alpha")
    if register:
        matplotlib.cm.register_cmap(name=name + "_alpha", cmap=cmap_alpha)
    return cmap_alpha

In [17]:
# Get our new version of the colormap with white2alpha
cmap_w2a = cmap_white2alpha("Greens")

In [18]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


# 4. Generating TIF Preview for the Top 18

In [19]:
def genAROItif_UwU(sample_ROI, sample_, name, mz, vmin_, vmax_):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    #mean         0.196373
    #std          0.180211
    #min          0.000000
    #25%          0.116014
    #50%          0.173645
    #75%          0.229128
    #max          2.152269
    #plt.imshow(df_HM, cmap=cmap_w2a, vmin=0.229128/0.5, vmax=2.152269*0.25, zorder=1)
    plt.imshow(df_HM, cmap=cmap_w2a, vmin=vmin_, vmax=vmax_, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_{name}_{mz}.tif", bbox_inches = 'tight', pad_inches = 0)

In [20]:
mzfeatures = ["311.1684", "269.2486", "215.0328", "295.2278", "883.5331", 
              "309.1704", "738.5059", "435.2965", "250.1449", "514.2846"]

In [21]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/raMSI_inFNA_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(371480, 2554)

In [22]:
raMSI_ML = df_raMSI_ML.reset_index()

In [23]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [24]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [25]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [26]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [27]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2]

In [29]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

# = 0
for TOP in list(mzfeatures):
    
    #vmax_ = vmax_list[v]
    counter = 0
    #start = datetime.now()
    #for df in ROI_sample_list:
    for df in sample_list:
        start = datetime.now()
        
        name = name_list[counter]
        sample_ = sample_list[counter]
        preview = sample_list_[counter][[TOP]]
        for i in preview.columns:
            preview[i] = preview[i].astype(float)
        vmax_list = list(preview.describe().iloc[-1])
        #vmin_list = list(preview.describe().iloc[-3])
        #vmax_list = list(preview.describe().iloc[-2])
        #vmin_ = vmin_list[0] * 0.25
        vmin_ = 0
        vmax_ = vmax_list[0]*0.8
        #print(TOP, name, vmax_)
        
        genAROItif_UwU(sample_ROI=sample_list_[counter], sample_=sample_, name=name, mz=TOP, vmin_=vmin_, vmax_=vmax_)
        #print(df.shape)
        counter += 1
        end = datetime.now()
        Duration = end - start
        print(Duration)


0:00:59.327753
0:01:07.812389
0:01:10.837852
0:00:35.908051
0:00:40.721196
0:00:55.630874
0:00:03.583113
0:00:02.694918
0:00:03.941093
0:00:03.785461
0:02:00.711618
0:01:54.940312
0:02:04.196700
0:00:57.677147
0:00:48.649047
0:00:41.694057
0:00:40.662544
0:00:39.512091
0:00:40.496096
0:00:02.567257
0:00:06.081238
0:00:03.368589
0:00:02.895077
0:01:50.722303
0:01:53.028459
0:02:07.153227
0:00:49.380019
0:00:55.080804
0:01:13.803688
0:00:41.879132
0:02:09.894712
0:06:58.798272
0:00:03.601186
0:00:02.623723
0:00:03.944908
0:00:03.814238
0:09:45.201131
0:02:19.774746
0:02:11.820572
0:00:56.755682
0:00:48.277616
0:00:40.837126
0:00:40.610279
0:00:39.598563
0:00:40.336530
0:00:02.524667
0:00:06.063204
0:00:03.494441
0:00:02.875492
0:01:48.017089
0:01:51.783968
0:02:04.893552
0:00:49.792823
0:00:54.522770
0:01:02.832850
0:00:34.684547
0:00:40.746166
0:00:55.126071
0:00:03.593700
0:00:02.517672
0:00:04.006258
0:00:03.986625
0:01:57.431939
0:02:03.796112
0:11:08.712381
0:05:09.042811
0:01:00.10

# 5. TIF Stacking

In [30]:
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)

new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)

new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)

new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
        
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
        
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
        
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
        
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/ROI_HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)

new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/ROI/merge"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    

In [118]:
for folders in glob.glob(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\ROI"):
    for filename in os.listdir(folders):
        sample = ("_").join(filename.split("\\")[-1].split("_")[:-1])
        #print(sample)
        if sample == "ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2":
            try:
                source = os.path.join(folders, filename)
                target = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\ROI\ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"
                target = os.path.join(target, filename)
                shutil.move(source, target)

            except:
                print("done")
                pass

In [119]:
import glob
import tifffile

with tifffile.TiffWriter(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\ROI\ROIstack_HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2.tif") as stack:
    for filename in glob.glob(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\ROI\ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2\*.tif"):
        stack.save(
            tifffile.imread(filename), 
            contiguous=True
        )

# 6. TIF Merging

In [44]:
def TIFMerger(TIFStack, TIFMerge):
    ret, images = cv2.imreadmulti(TIFStack, [], cv2.IMREAD_UNCHANGED)
    alp = 0.9
    img1 = images[0]
    img2 = images[1]
    added_image = cv2.addWeighted(img1, alp, img2, alp, 0)
    for i in range(2, 10):
        img = images[i]
        added_image = cv2.addWeighted(added_image, alp, img, alp**i, 0)
        cv2.imwrite(TIFMerge, added_image)

In [120]:
TIFMerger(
    TIFStack= r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\ROI\ROIstack_HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2.tif", 
    TIFMerge= r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\ROI\merge\ROImerge_HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2.tif"
)